In [1]:
import pandas as pd
DelDF = pd.read_csv('Deliveries.csv')
OldDelDF = pd.read_csv('DeliveriesOld.csv')
DaysDF = pd.read_csv('Days.csv')
WeeksDF = pd.read_csv('Weeks.csv')
gpsDF = pd.read_csv('GPS.csv')

In [2]:
def dollars_to_floats(df, columns):
    for x in columns:
        print(x)
        df[x] = df[x].str.replace('$', '')
        df[x] = df[x].str.replace(',', '')
        df[x] = df[x].astype(float)

In [3]:
frames = [OldDelDF, DelDF]
result = pd.concat(frames)
#We need to reset the index after the combination of v1/v2, and v3
result.reset_index(drop=True, inplace=True)

#DelDF is v3
#results is all deliveries
#Those are the only 2 we need

In [ ]:
result.dtypes

In [4]:
#Use function for pay, tip, total, peak bonus
result["Pay"] = result["Pay"].str.replace('$', '')
result["Tip"] = result["Tip"].str.replace('$', '')
result["Total"] = result["Total"].str.replace('$', '')
result["Peak Bonus"] = result["Peak Bonus"].str.replace('$', '')

result["Total"] = result["Total"].astype('float')
result["Pay"] = result["Pay"].astype('float')
result["Tip"] = result["Tip"].astype('float')
result["Peak Bonus"] = result["Peak Bonus"].astype('float')

/tmp/ipykernel_41442/1330755715.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  result["Pay"] = result["Pay"].str.replace('$', '')
/tmp/ipykernel_41442/1330755715.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  result["Tip"] = result["Tip"].str.replace('$', '')
/tmp/ipykernel_41442/1330755715.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  result["Total"] = result["Total"].str.replace('$', '')
/tmp/ipykernel_41442/1330755715.py:4: FutureWarning: The default value of regex will change from True to False in a f

In [7]:
#Doodle to get the percentage of orders that tip
numUntipped = (result["Tip"] == 0.0).sum()
numTipped = (result["Tip"] != 0.0).sum()
percTipped = (numTipped/(numUntipped+numTipped))
print(percTipped)

77
597
0.8857566765578635


In [8]:
# 88.58% of customers tip

In [9]:
#Doodle that converts 'Date' attribute to Timestamp object

from datetime import date, time

datevar = result.iloc[600]["Date"]
timevar = result.iloc[600]["Start Time"]
print(datevar, " ", timevar)
datevar1 = '2023/' + datevar + " " + timevar
datevar1 = pd.Timestamp(datevar1)
print(datevar1)
print(datevar1.day_name())
print(type(datevar1))

5/27   7:19:00 PM
2023-05-27 19:19:00
Saturday
<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [11]:
#Only convert v3 to datetime, except Date which can all be converted
datetest = result.iloc[600]["Date"] + "/2023"
timetest = result.iloc[600]["Start Time"]
datetest = pd.to_datetime(datetest, format="%m/%d/%Y")
print(type(datetest)) #this is a Timestamp and note datetime object?
print(datetest.strftime('%m-%d'), timetest)

#So maybe what we want to do, is convert date, start, end to datetime, exclude v1/v2 for start/end related analysis
#now try convert start time to date time
timetest1 = pd.to_datetime(timetest)
print(timetest1)

#interesting, it automatically uses the current date without an existing date
#maybe we just convert both to datetime, then use datetime.date for one and datetime.time for the other

print(timetest1.hour, timetest1.minute, datetest.month, datetest.day)
datetimetest = pd.Timestamp.combine(date(datetest.year, datetest.month, datetest.day), time(timetest1.hour, timetest1.minute))
print(datetimetest)
print(type(datetimetest))

#Okay so that's how we can get the timestamp for date and for time, that leaves miles and duration to check

<class 'pandas._libs.tslibs.timestamps.Timestamp'>
05-27 7:19:00 PM
2023-06-20 19:19:00
19 19 5 27
2023-05-27 19:19:00
<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [13]:
# GPS DOODLE HERE
# Initial doodle to help with address entry
# Need to rework to handle both CSVs now
#gpsDF1 = gpsDF[["ID", "Date", "Restaurant Name", "RAW DATA"]].copy()
#gpsDF2 = gpsDF1[["Restaurant Name", "RAW DATA"]].copy()
#gpsDF2 = gpsDF2.drop_duplicates(subset=["Restaurant Name", "RAW DATA"])
#gpsDF2 = gpsDF2.sort_values(by="Restaurant Name", ascending=True)
#gpsDF2.to_csv('gpsDF2.csv')

,Restaurant Name,RAW DATA
0,Erika's Restaurant,Erika's Mexican Food & Seafood Vista
1,Chick-fil-A,Chick-fil-A (01507)
2,Wendy's,Wendy's (8407)
3,Annam Pho and Sandwiches,Annam Pho and Sandwiches (Plaza Dr)
4,Disfruta,Disfruta (Oceanside)
...,...,...
688,Big Bob's Best Pizza,Big Bob's Best Pizza
696,???(tacobell)???,CCC0005121 (3845 Plaza Drive)
703,Del Taco,Del Taco (1305)
704,??(other taco bell),CCC0005121 (695 College Boulevard)


In [15]:
#Why not try
#result[result["Peak Bonus"] != 0 ].shape

tempDF = result[result["Peak Bonus"] != 0 ]
tempDF.shape
#Since a significant volume of orders are off peak
#(because peak happens about 1am-3am)
#And since I dont really want to spend lots of time working those hours
#It might make sense to re-evaluate the pursuit of peak pay based insights

(45, 13)

In [22]:
#tempDF = result[result["Restaurant Name"].duplicated(keep=False) == False]
#This is all the places I've picked up from only once
#duplicated(keep=False) flags all rows that appear more than once to be flagged as duplicates instead of default
#default is all rows after the first flagged as duplicates

In [23]:
tempDF = result[result["Restaurant Name"].duplicated(keep=False) == True]
tempDF = tempDF.groupby(["Restaurant Name"]).agg({"Pay": "mean", "Tip": "mean"})
tempDF = tempDF.round({'Pay': 2, 'Tip': 2})
tempDF.sort_values("Tip", ascending=False)

In [25]:
#So we see car parts and flower orders as the only actual 0 tip
#On the other hand, grocery orders tip well, which makes sense because they are much more work than normal
#And we see large avg tip values from two pizza places in the top 5
#I think it would be more useful to have a count as a 4th column

In [26]:
#Overall hourly rate, Total from TOTALS / Dash
#Hardcoded index for Totals, should look for row where dates == "TOTALS" 
totalPay = WeeksDF.loc[10]["Total"]

totalPay = totalPay.replace('$', '')
totalPay = totalPay.replace(',', '')
totalPay = float(totalPay)

totalTime = WeeksDF.loc[10]["Dash"]
totalTimeH = totalTime[0:3]
totalTimeM = totalTime[4:6]
#okay so it includes the first position and does not include the second position, so its a [x, y) situation
totalTimeH = float(totalTimeH)
totalTimeM = float(totalTimeM)
totalTimeM = totalTimeM/60
totalTime = totalTimeH + totalTimeM

hourlyRate = totalPay/totalTime
print("$", hourlyRate, "per hour.")

$7,112.19
7112.19
59.0
0.9833333333333333
294.98333333333335
$ 24.110480818125314 per hour.


In [27]:
#So float(var) converts, but series.astype(float) is for series in df

/tmp/ipykernel_41442/1848943084.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  WeeksDF["Pre Total"] = WeeksDF["Pre Total"].str.replace('$', '')


,Dates,Base,Tip,Adj pay,Other,Pre Total,Total,Active,Dash,Deliveries,Version
0,3/27 - 4/2,$108.50,$88.00,$40.96,$0.00,196.50,$237.46,6:42:00,10:55:00,22,1.0
1,4/3 - 4/9,$264.25,$230.69,$58.17,$0.00,494.94,$553.11,14:41:00,22:02:00,55,1.0
2,4/10 - 4/16,$130.75,$96.13,$61.07,$0.00,226.88,$287.95,8:38:00,11:10:00,28,1.0
3,4/17 - 4/23,$382.50,$248.41,$105.05,$0.00,630.91,$735.96,21:20:00,32:18:00,68,1.0
4,4/24 - 4/30,$430.00,$299.73,$138.76,$2.62,729.73,$871.11,25:30:00,41:10:00,84,1.0
5,5/1 - 5/7,$618.50,$445.32,$121.74,$0.00,1063.82,"$1,185.56",32:24:00,50:40:00,114,1.0
6,5/8 - 5/14,$758.25,$567.84,$155.48,$6.25,1326.09,"$1,487.82",40:35:00,58:08:00,138,2.0
7,5/15 - 5/21,$303.75,$239.25,$99.27,$0.00,543.00,$642.27,17:32:00,25:24:00,64,2.0
8,5/22 - 5/28,$205.50,$189.19,$63.63,$0.00,394.69,$458.32,11:39:00,16:40:00,38,3.0
9,5/29 - 6/4,$342.75,$309.88,$115.30,$0.00,652.63,$767.93,19:24:00,26:32:00,63,3.0


In [29]:
#Doodle to 
#convert $#.## to float and hh:mm:ss to float, 
#then make the pre total to hours worked ratio, then sort and show
#Should change this to use function dollars_to_floats
WeeksDF["Pre Total"] = WeeksDF["Pre Total"].str.replace('$', '')
WeeksDF["Pre Total"] = WeeksDF["Pre Total"].str.replace(',', '')
WeeksDF["Pre Total"] = WeeksDF["Pre Total"].astype(float)

for index, row in WeeksDF.iterrows():
    active = row["Active"]
    activeH = float(active[:active.index(":")])
    activeM = round(float(active[active.index(":")+1:active.index(":")+3])/60, 2)
    active = activeH + activeM
    WeeksDF.at[index, "Active"] = active
#Okay now we have pre total and active in a format that allows for good math
WeeksDF["pretotalactiveratio"] = WeeksDF["Pre Total"]/WeeksDF["Active"]
WeeksDF.sort_values(by=["pretotalactiveratio"])
#right well its kind of interesting to see totals in the middle
#and it also looks like May had better ratios, which makes it seem like I'm getting better?
#better isn't a good characterization
#pre total is the money before adjustment, its just base pay + tip 
#so may having better base pay + tip / time actually spent on deliveries is interesting

In [32]:
#Dollars Earned vs Miles Driven doodle
#$ to float
#drop non numeric Miles values, then convert column to float
#agg sum Total and Miles, by restaurant name (then round)
#create new column as Total/Miles
#sort low to high
v3DF = DelDF.copy()
v3DF["Total"] = v3DF["Total"].str.replace('$', '')
v3DF["Total"] = v3DF["Total"].str.replace(',', '')
v3DF["Total"] = v3DF["Total"].astype(float)
v3DF = v3DF[v3DF.Miles!="-"]
v3DF["Miles"] = v3DF["Miles"].astype(float)
v3DF = v3DF.groupby(["Restaurant Name"]).agg({"Total": "sum", "Miles": "sum"})
v3DF = v3DF.round({'Pay': 2, 'Tip': 2})
v3DF["DollarMilesRatio"] = v3DF["Total"]/v3DF["Miles"]
v3DF.sort_values(by="DollarMilesRatio")

/tmp/ipykernel_41442/1666305466.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  v3DF["Total"] = v3DF["Total"].str.replace('$', '')


,Total,Miles,DollarMilesRatio
Restaurant Name,,,
Urban Plates,21.50,20.40,1.053922
7-Eleven,20.00,18.40,1.086957
Colimas Mexican Food,6.25,5.30,1.179245
Gianni's Pizza,11.50,9.50,1.210526
Dashmart,9.75,7.60,1.282895
Jack in the Box,20.25,15.50,1.306452
Pizza on tap,13.50,10.10,1.336634
China Fusion,39.00,26.40,1.477273
Felix's BBQ With Soul,20.50,13.80,1.485507


In [33]:
DaysDF.max() #would be useful with more attributes converted to float 

Date                    6/4
Total                $86.13
Base                 $99.00
Tip                  $94.03
Start (24)            18:58
End (24)               2:32
Active time (h:m)      9:40
Dash time (h:m)        9:41
Offers                   32
Deliveries               33
Version                   3
Pauses                    2
dtype: object

In [35]:
#Avg tip vs Avg tip excluding zero tip orders

TipAvg = result["Tip"].mean()
print(TipAvg)
TipAvg2 = result[result["Tip"] !=0]["Tip"].mean()
print(TipAvg2)

4.027359050445104
4.54680067001675


In [36]:
#doodle checking if dollars_to_floats works
tempDF = DelDF.copy()
listToPass = ["Total", "Pay", "Tip", "Peak Bonus"]
dollars_to_floats(tempDF, listToPass)
#Wanted to check Miles to see which non numeric values we had to drop
tempDF['Miles'].unique()

Total
Pay
Tip
Peak Bonus


/tmp/ipykernel_41442/57653493.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[x] = df[x].str.replace('$', '')


array(['6.2', '2.9', '5.5', '3.1', '3.3', '7.1', '4.84', '5.2', '3.7',
       '4.7', '3.0', '5.9', '5.1', '-', '8.2', '6.1', '7.2', '3.2', '2.6',
       '2.4', '9.5', '8.3', '2.3', '6.9', '4.5', '7.7', '2.5', '5.0',
       '1.8', '4.0', '7.3', '6.3', '5.3', '7.5', '2.0', '3.4', '3.6',
       '4.2', '6.0', '2.7', '7.4', '10.1', '8.0', '15.9', '7.6', '6.5',
       '4.4', '1.9', '4.3', '4.1', '1.4', '4.8', '1.2', '6.8', '10.2',
       '12.1'], dtype=object)

In [37]:
#tempDF doodle
#Drop the non numeric values (which are just '-') and convert Miles to float from string
tempDF = tempDF[tempDF['Miles'] != '-']
tempDF['Miles'] = tempDF['Miles'].astype(float)

In [40]:
#DATETIME DOODLE

date = tempDF.iloc[-1]["Date"]
starttime = tempDF.iloc[-1]["Start Time"]
endtime = tempDF.iloc[-1]["End Time"]
print(date, starttime, endtime)
date = '2023/' + datevar + " " + starttime
date = pd.Timestamp(date)
starttime = date
endtime = '2023/' + datevar + " " + endtime
endtime = pd.Timestamp(endtime)
print("\n", date)
print(date.day_name())
print(starttime.minute)
print(endtime.minute)

6/4 6:21:00 PM 6:51:00 PM

 2023-05-27 18:21:00
Saturday
21
51


In [ ]:
#datetest = result.iloc[600]["Date"] + "/2023"
#timetest = result.iloc[600]["Start Time"]
#datetest = pd.Timestamp(datetest)
#datetest = pd.to_datetime(datetest, format="%m/%d/%Y")
#print(datetest.strftime('%m-%d'), timetest)


#timetest1 = pd.to_datetime(timetest)

#datetimetest = pd.Timestamp.combine(date(datetest), time(timetest1))
#print(timetest1.hour, timetest1.minute, datetest.month, datetest.day)
#datetimetest = pd.Timestamp.combine(date(datetest.year, datetest.month, datetest.day), time(timetest1.hour, timetest1.minute))
#print(datetimetest)

In [41]:
#Doodle of loop that adds 2023 to the date string to allow for easier conversion to datetime object
for index, row in DaysDF.iterrows():
    date = pd.Timestamp('2023/' + row["Date"])
    DaysDF.at[index, "Date"] = date
    #the below should work instead of the line outside the loop
    #DaysDF.at[index, "Date"] = pd.to_datetime(date)
    #or = pd.Timestamp(date)
#Converts date from string to datetime
DaysDF["Date"] = pd.to_datetime(DaysDF["Date"])

In [42]:
#Adds column to DaysDF for the day of the week (Monday, Thursday, etc.)
DaysDF["Day of Week"] = DaysDF["Date"].dt.day_name()

In [43]:
#Doodle of for loop and type cast to convert from hh:mm to float
#This should be made into a function
for index, row in DaysDF.iterrows():
    #print(row["Active"])
    active = row["Dash time (h:m)"]
    activeH = float(active[:active.index(":")])
    activeM = round(float(active[active.index(":")+1:active.index(":")+3])/60, 2)
    active = activeH + activeM
    #print(active)
    DaysDF.at[index, "Dash time (h:m)"] = active
#Converts the string value for hours worked into float
DaysDF["Dash time (h:m)"] = DaysDF["Dash time (h:m)"].astype(float)

In [46]:
#Converts the string value for hours worked into float
DaysDF["Dash time (h:m)"] = DaysDF["Dash time (h:m)"].astype(float)
#Converts Total, Base, and Tip attributes to float using the dollars_to_floats func
dollars_to_floats(DaysDF, ["Total", "Base", "Tip"])
#Gets the total amount of hours worked, total pay, total tips earned, and total deliveries by day of week
DaysDF = DaysDF.groupby(["Day of Week"]).agg(
    {"Dash time (h:m)": "sum", "Total": "sum", "Tip": "sum", "Deliveries ": "sum"})
DaysDF = DaysDF.round({'Dash time (h:m)': 2, 'Total': 2, "Tip": 2})
#Creates a new column to get the hourly pay rate by day of week
DaysDF["$/hr"] = DaysDF["Total"]/DaysDF["Dash time (h:m)"]
#Creates a new column to get the average tip per order by day of the week
DaysDF["AvgTip"] = DaysDF["Tip"]/DaysDF["Deliveries "]
display(DaysDF)